In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import mutual_info_regression

# Library data visualization
import plotly
import matplotlib.pylab as pl
import matplotlib as m
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib import pyplot as plt
from IPython.display import SVG

from scipy import stats

In [60]:
file_path = "Datasets\solar_sensor_data.csv" 
sensor_df = pd.read_csv(file_path)

In [61]:
test_file_path = "Datasets\weather_sensor_data.csv"
weather_df = pd.read_csv(test_file_path)

In [62]:
sensor_df

,LOCATION,DATE_TIME,SENSOR_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,A,15-05-2020 00:00,sensor 1,0.0,0.0,0.0,6259559.0
1,A,15-05-2020 00:00,sensor 2,0.0,0.0,0.0,6183645.0
2,A,15-05-2020 00:00,sensor 3,0.0,0.0,0.0,6987759.0
3,A,15-05-2020 00:00,sensor 5,0.0,0.0,0.0,7602960.0
4,A,15-05-2020 00:00,sensor 12,0.0,0.0,0.0,7158964.0
...,...,...,...,...,...,...,...
136471,B,2020-06-17 23:45:00,sensor 33,0.0,0.0,4157.0,520758.0
136472,B,2020-06-17 23:45:00,sensor 35,0.0,0.0,3931.0,121131356.0
136473,B,2020-06-17 23:45:00,sensor 38,0.0,0.0,4322.0,2427691.0
136474,B,2020-06-17 23:45:00,sensor 40,0.0,0.0,4218.0,106896394.0


In [63]:
weather_df

,LOCATION,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,A,2020-05-15 00:00:00,25.184316,22.857507,0.0
1,A,2020-05-15 00:15:00,25.084589,22.761668,0.0
2,A,2020-05-15 00:30:00,24.935753,22.592306,0.0
3,A,2020-05-15 00:45:00,24.846130,22.360852,0.0
4,A,2020-05-15 01:00:00,24.621525,22.165423,0.0
...,...,...,...,...,...
6436,B,2020-06-17 22:45:00,23.511703,22.856201,0.0
6437,B,2020-06-17 23:00:00,23.482282,22.744190,0.0
6438,B,2020-06-17 23:15:00,23.354743,22.492245,0.0
6439,B,2020-06-17 23:30:00,23.291048,22.373909,0.0


In [64]:
def customDescription(df: pd.DataFrame, numeric_only: bool = False):
    if numeric_only:
        df = df.select_dtypes(include=np.number)
    
    desc = pd.DataFrame(index=df.columns.to_list())
    desc['type'] = df.dtypes
    desc['count'] = df.count()
    desc['nunique'] = df.nunique()
    desc['null'] = df.isnull().sum()
    
    # Handle numeric columns separately
    if not df.select_dtypes(include=np.number).empty:
        numeric_desc = df.describe().T.drop(columns=['count','std','25%','50%','75%'], axis=1)
        for col in df.select_dtypes(include=np.number).columns:
            desc.loc[col, 'mean'] = numeric_desc.loc[col, 'mean']
            desc.loc[col, 'min'] = numeric_desc.loc[col, 'min']
            desc.loc[col, 'max'] = numeric_desc.loc[col, 'max']
    
    # Handle non-numeric columns separately
    if not df.select_dtypes(exclude=np.number).empty:
        non_numeric_desc = df.select_dtypes(exclude=np.number).describe().T
        desc = pd.concat([desc, non_numeric_desc.drop(columns=['count', 'unique', 'top', 'freq'], axis=1, errors='ignore')], axis=1)
    
    return desc

In [65]:
customDescription(sensor_df)

,type,count,nunique,null,mean,min,max
LOCATION,object,136476,2,0,NaN,NaN,NaN
DATE_TIME,object,136476,6417,0,NaN,NaN,NaN
SENSOR_ID,object,136476,44,0,NaN,NaN,NaN
DC_POWER,float64,136476,63581,0,1.708541e+03,0.0,1.447112e+04
AC_POWER,float64,136476,62872,0,2.748035e+02,0.0,1.410950e+03
DAILY_YIELD,float64,136476,59249,0,3.295434e+03,0.0,9.873000e+03
TOTAL_YIELD,float64,136476,70381,0,3.303821e+08,0.0,2.247916e+09


In [66]:
customDescription(weather_df)

,type,count,nunique,null,mean,min,max
LOCATION,object,6441,2,0,NaN,NaN,NaN
DATE_TIME,object,6441,3262,0,NaN,NaN,NaN
AMBIENT_TEMPERATURE,float64,6441,6441,0,26.815672,20.398505,39.181638
MODULE_TEMPERATURE,float64,6441,6441,0,31.941762,18.140415,66.635953
IRRADIATION,float64,6441,3620,0,0.230551,0.000000,1.221652


In [67]:
# Reset SENSOR_ID
# sensor_df['SENSOR_ID'] = range(1, len(sensor_df) + 1)

In [68]:
# Convert DATE_TIME to the desired format only where LOCATION is "A"
sensor_df.loc[sensor_df['LOCATION'] == "A", 'DATE_TIME'] = pd.to_datetime(
    sensor_df.loc[sensor_df['LOCATION'] == "A", 'DATE_TIME'], 
    format='%d-%m-%Y %H:%M'
).dt.strftime('%Y-%m-%d %H:%M:%S')

In [54]:
# Merge the sensor and weather DataFrames on LOCATION and DATE_TIME
merged_df = pd.merge(sensor_df, weather_df, on=["LOCATION", "DATE_TIME"], how="inner")

In [43]:
# Split the DataFrame into two based on LOCATION
df_A = merged_df[merged_df['LOCATION'] == "A"].reset_index(drop=True)
df_B = merged_df[merged_df['LOCATION'] == "B"].reset_index(drop=True)

In [73]:
sensor_df

,LOCATION,DATE_TIME,SENSOR_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,A,2020-05-15 00:00:00,1,0.0,0.0,0.0,6259559.0
1,A,2020-05-15 00:00:00,2,0.0,0.0,0.0,6183645.0
2,A,2020-05-15 00:00:00,3,0.0,0.0,0.0,6987759.0
3,A,2020-05-15 00:00:00,4,0.0,0.0,0.0,7602960.0
4,A,2020-05-15 00:00:00,5,0.0,0.0,0.0,7158964.0
...,...,...,...,...,...,...,...
136471,B,2020-06-17 23:45:00,136472,0.0,0.0,4157.0,520758.0
136472,B,2020-06-17 23:45:00,136473,0.0,0.0,3931.0,121131356.0
136473,B,2020-06-17 23:45:00,136474,0.0,0.0,4322.0,2427691.0
136474,B,2020-06-17 23:45:00,136475,0.0,0.0,4218.0,106896394.0
